# Introdução

# *Imports* e Funções

In [ ]:
import os
import shutil
import zipfile
import requests
import geopandas as gpd
from bs4 import BeautifulSoup

In [ ]:
path = 'data/brutos'
os.makedirs(path, exist_ok=True)

Função para fazer *download* usando o *request*. Ainda, a função pega o nome do arquivo a partir do *Content Disposition*. Usei a função do *post* [Downloading Files from URLs in Python](https://www.codementor.io/@aviaryan/downloading-files-from-urls-in-python-77q3bs0un).

In [ ]:
import requests
import re

def get_filename_from_cd(cd):
    """
    Get filename from content-disposition
    """
    if not cd:
        return None
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    return fname[0]

# Dados Espaciais

## Limite Municipal SP (IGC)

In [ ]:
# Metadados
url = 'http://datageo.ambiente.sp.gov.br/geoportal/catalog/search/resource/details.page?uuid='
id_metadados = '{74040682-561A-40B8-BB2F-E188B58088C1}'

print(url+id_metadados)

In [ ]:
# Abre a página dos metadados
page = requests.get(url+id_metadados)
print('Resposta da página foi {}'.format(page))

# Parser HTML
soup = BeautifulSoup(page.content, 'html.parser')
soup = soup.find_all('a', href=True)

# Procura Shapefile
for i in soup:
    text = i.text.split(' ')
    #print(text)
    for j in text:
        #print(j)
        if j in 'Shapefile':
            print('> Encontrei o shapefile')
            url = i['href']
            print('Link: {}'.format(url))

In [ ]:
# Download file in data/brutos
r = requests.get(url, allow_redirects=True)
filename = get_filename_from_cd(r.headers.get('content-disposition'))
open(os.path.join('data', 'brutos', filename), 'wb').write(r.content)

In [ ]:
# Unzip
file = os.path.join('data', 'brutos', filename)
temp = os.path.join(os.path.dirname(file), 'temp')
os.makedirs(temp, exist_ok=True)

with zipfile.ZipFile(file, 'r') as zip_ref:
    zip_ref.extractall(temp)

In [ ]:
# Lista Arquivos
os.listdir(temp)

In [ ]:
# Read shapefile
gdf = gpd.read_file(os.path.join(temp, 'LimiteMunicipalPolygon.shp'))
display(gdf.head(5))
gdf.plot()

In [ ]:
# Renomeia Colunas
gdf = gdf.rename(columns={'Cod_ibge':'ID_IBGE',
                          'Nome':'Nome_Municipio',
                          'Rotulo':'Rotulo_Municipio'})

# Deleta Colunas
gdf = gdf.drop(['Cod_Cetesb', 'UGRHI', 'Nome_ugrhi'], axis=1)

print(gdf.dtypes)
display(gdf.head(5))

In [ ]:
# Reprojeta
print(gdf.crs)
gdf = gdf.to_crs(epsg=4326)
print(gdf.crs)
gdf.plot()

In [ ]:
# Salva
gdf.to_file(os.path.join('data', 'LimiteMunicipal.geojson'), driver='GeoJSON', encoding='utf-8')
gdf.to_file(os.path.join('data', 'LimiteMunicipal' + '.gpkg'), layer='Limite', driver="GPKG")

In [ ]:
# Excluí pasta temporária
shutil.rmtree(temp)

## Sedes Municipais

In [ ]:
# Metadados
url = 'http://datageo.ambiente.sp.gov.br/geoportal/catalog/search/resource/details.page?uuid='
id_metadados = '{64BF344A-3AD0-410A-A3AA-DFE01C4E9BBB}'

print(url+id_metadados)

In [ ]:
# Abre a página dos metadados
page = requests.get(url+id_metadados)
print('Resposta da página foi {}'.format(page))

# Parser HTML
soup = BeautifulSoup(page.content, 'html.parser')
soup = soup.find_all('a', href=True)

# Procura Shapefile
for i in soup:
    text = i.text.split(' ')
    #print(text)
    for j in text:
        #print(j)
        if j in 'Shapefile':
            print('> Encontrei o shapefile')
            url = i['href']
            print('Link: {}'.format(url))

In [ ]:
# Download file in data/brutos
r = requests.get(url, allow_redirects=True)
filename = get_filename_from_cd(r.headers.get('content-disposition'))
open(os.path.join('data', 'brutos', filename), 'wb').write(r.content)

In [ ]:
# Unzip
file = os.path.join('data', 'brutos', filename)
temp = os.path.join(os.path.dirname(file), 'temp')
os.makedirs(temp, exist_ok=True)

with zipfile.ZipFile(file, 'r') as zip_ref:
    zip_ref.extractall(temp)

In [ ]:
# Lista Arquivos
os.listdir(temp)

In [ ]:
# Read shapefile
gdf = gpd.read_file(os.path.join(temp, 'SedesMunicipaisPoint.shp'))
display(gdf.head(5))
gdf.plot()

In [ ]:
# Renomeia Colunas
gdf = gdf.rename(columns={'Nome':'Nome_Municipio'})

# Deleta Colunas
gdf = gdf.drop(['Codigo_CET'], axis=1)

print(gdf.dtypes)
display(gdf.head(5))

In [ ]:
# Reprojeta
print(gdf.crs)
gdf = gdf.to_crs(epsg=4326)
print(gdf.crs)
gdf.plot()

In [ ]:
# Salva
gdf.to_file(os.path.join('data', 'SedesMunicipais.geojson'), driver='GeoJSON', encoding='utf-8')
gdf.to_file(os.path.join('data', 'SedesMunicipais' + '.gpkg'), layer='Sedes', driver="GPKG")

In [ ]:
# Excluí pasta temporária
shutil.rmtree(temp)

# Git

In [ ]:
%run '../codes/git/update_github.py'

git_full('.', '.', 'Criação do repo')